# 텐서와 연산

tensorflow 2.x부터는 즉시 실행(eager execution)이 기본 실행

In [1]:
import tensorflow as tf

## 텐서
텐서는 다차원 배열입니다. 넘파이(NumPy) ndarray 객체와 비슷하며, tf.Tensor 객체는 데이터 타입과 크기를 가지고 있습니다. 또한 tf.Tensor는 GPU 같은 가속기 메모리에 상주할 수 있습니다. 텐서플로는 텐서를 생성하고 이용하는 풍부한 연산 라이브러리(tf.add, tf.matmul, tf.linalg.inv 등.)를 제공합니다. 이러한 연산은 자동으로 텐서를 파이썬 네이티브(native) 타입으로 변환합니다.

In [2]:
print(tf.add(1,2))
print(tf.add([1, 2], [3,4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# 연산자 오버로딩 지원
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [3]:
x = tf.matmul([[1]], [[2,3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


### 넘파이 호환성
* tensorflow는 numpy 배열을 자동으로 텐서로 변환
* numpy 연산은 자동으로 텐서를 numpy array로 변환

In [8]:
import numpy as np

ndarray = np.ones([3, 3])

print("tensorflow 연산은 자동으로 numpy 배열을 텐서로 변환")
tensor = tf.math.multiply(ndarray, 42)
print(tensor)

print("numpy 연산은 자동으로 텐서를 numpy 배열로 변환")
print(np.add(tensor, 1))

print(".numpy() 메서드는 텐서를 numpy 배열로 변환")
print(tensor.numpy())

tensorflow 연산은 자동으로 numpy 배열을 텐서로 변환
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
numpy 연산은 자동으로 텐서를 numpy 배열로 변환
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
.numpy() 메서드는 텐서를 numpy 배열로 변환
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU 가속
텐서플로우는 연산을 위해 CPU 또는 GPU를 사용할 것인지 자동으로 결정

In [9]:
x = tf.random.uniform([3, 3])

print("GPU 사용이 가능한가 : ")
print(tf.test.is_gpu_available())

print("텐서가 GPU #0에 있는가 : ")
print(x.device.endswith('GPU:0'))

GPU 사용이 가능한가 : 
True
텐서가 GPU #0에 있는가 : 
True


### 장치 이름
텐서를 구성하고 있는 호스트 장치의 풀네임을 제공. 텐서가 N번째 GPU에 있다면 GPU:N으로 반환.
### 명시적 장치 배치
Tensorflow는 연산 실행을 이해 장치를 자동으로 결정함. 사용자는 `tf.device`를 사용해 특정 장치에 명시적 배치 가능

In [11]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
    
    result = time.time() - start
    
    print("10 loops: {:0.2f}ms".format(1000*result))
    
# CPU에서 강제 실행
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# GPU #0가 이용 가능한 경우 GPU #0에서 강제 실행
if tf.test.is_gpu_available():
    print("On GPU:")
    with tf.device("GPU:0"):
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)    

On CPU:
10 loops: 75.07ms
On GPU:
10 loops: 493.45ms


## 데이터셋
이번에는 모델에 데이터를 제공하기 위한 파이프라인을 구축하기 위해 `tf.data.Dataset` API를 사용해볼 것입니다.

`tf.data.Dataset API`는 모델을 훈련시키고 평가 루프를 제공할, 간단하고 재사용 가능한 모듈로부터 복잡한 입력 파이프라인을 구축하기 위해 사용됩니다.

### 소스 데이터셋 생성
굉장히 유용한 함수중 하나인 `Dataset.from_tensors`, `Dataset.from_tensor_slices`와 같은 팩토리(factory) 함수 중 하나를 사용하거나 파일로부터 읽어들이는 객체인 `TextLineDataset` 또는 `TFRecordDataset`를 사용하여 소스 데이터셋을 생성하세요. 더 많은 정보를 위해서 텐서플로 데이터셋 가이드를 참조하세요.

In [18]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1,2,3,4,5,6])
print(ds_tensors)

# CSV 파일을 생성
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line 1
    Line 2
    Line 3
    """)
    
ds_file = tf.data.TextLineDataset(filename)

<TensorSliceDataset shapes: (), types: tf.int32>


### 변환 적용
`map`, `batch`, `shuffle`같은 변환 함수를 사용하여 데이터셋에 적용

In [19]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

### 반복
`tf.data.Dataset`은 레코드 순회를 지원하는 반복가능한 객체이다.

In [20]:
print('ds_tensors 요소: ')
for x in ds_tensors:
    print(x)

print('\nds_file 요소:')
for x in ds_file:
    print(x)

ds_tensors 요소: 
tf.Tensor([1 9], shape=(2,), dtype=int32)
tf.Tensor([16 25], shape=(2,), dtype=int32)
tf.Tensor([36  4], shape=(2,), dtype=int32)

ds_file 요소:
tf.Tensor([b'Line 1' b'    Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'    Line 3' b'    '], shape=(2,), dtype=string)
